# Physics-Informed Neural Network with Kolmogorov-Arnold Networks (PINN-KAN) for Allen–Cahn Equation


This notebook implements a complete **PINN–KAN pipeline** for solving the **Allen–Cahn equation**, a nonlinear PDE that models phase separation processes in materials science.

We use:
- **PINNs (Physics-Informed Neural Networks)** to incorporate physical laws into the training process,
- **KANs (Kolmogorov–Arnold Networks)** as flexible and interpretable neural function approximators,
- **Radial Basis Functions (RBFs)** for effective spatial feature representation.

---

### Notebook Structure

This notebook is organized into ten major stages:

1. **RBF Components** – Define radial basis kernels for spatial encoding  
2. **Model Architectures** – Build KAN and PINN models  
3. **Physics Residuals** – Formulate the Allen–Cahn PDE residual  
4. **Loss Functions** – Combine PDE, boundary, and initial condition losses  
5. **Training** – Optimize the model using gradient-based methods  
6. **Evaluation** – Assess model accuracy and residual consistency  
7. **Visualization** – Compare predicted vs. true field solutions  
8. **Main Pipeline** – Integrate all steps into a unified workflow  
9. **Run Experiment** – Execute multiple configurations for comparison  
10. **Diagnostics & Summary** – Analyze, compare, and interpret final results  




In [30]:
"""
Complete PINN-KAN Pipeline for Allen-Cahn Equation
Includes: PINN-KAN, Vanilla MLP, Vanilla PINN
With comprehensive visualization and metrics
"""

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Optional
import os
import pickle
from scipy.interpolate import griddata
import logging

# Configure a module-level logger and ensure we don't add multiple handlers
logger = logging.getLogger("pinn_logger")
if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter("%(message)s")
    handler.setFormatter(formatter)
    logger.addHandler(handler)
logger.setLevel(logging.INFO)

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


## Step 1: Radial Basis Function (RBF) Components
We begin by defining the RBF kernels that help represent the spatial structure of the Allen–Cahn field.

These functions will later be used to build flexible spatial embeddings for the model inputs.

**Key Concepts:**
- RBFs are used to map spatial inputs into higher-dimensional feature spaces.
- Useful for capturing local patterns and smooth variations.


In [31]:
# ==================== RBF COMPONENTS ====================

class RBFEdge(nn.Module):
    """Gaussian Radial Basis Function layer."""
    def __init__(self, input_dim: int, num_rbfs: int):
        super().__init__()
        self.centers = nn.Parameter(torch.randn(num_rbfs, input_dim))
        self.sigmas = nn.Parameter(torch.ones(num_rbfs, input_dim))
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        expanded = (x.unsqueeze(1) - self.centers) ** 2
        scaled = expanded / (2 * (self.sigmas ** 2))
        return torch.exp(-scaled.sum(dim=-1))


class KANLayer(nn.Module):
    """
    KANLayer: maps from input_dim -> output_dim using an RBF expansion
    num_rbfs controls the size of the intermediate RBF feature map.
    """
    def __init__(self, input_dim: int, num_rbfs: int, output_dim: int):
        super().__init__()
        self.rbf_edge = RBFEdge(input_dim=input_dim, num_rbfs=num_rbfs)  # centers sigma shape (num_rbfs, input_dim)
        self.linear = nn.Linear(num_rbfs, output_dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x shape: (N, input_dim)
        phi = self.rbf_edge(x)   # -> (N, num_rbfs)
        return self.linear(phi)  # -> (N, output_dim)


## Step 2: Model Architectures
In this section, we define the model architectures for both:
- **Kolmogorov–Arnold Network (KAN)** layers  
- **Physics-Informed Neural Network (PINN)** components  

These models approximate the solution to the PDE while embedding physics constraints directly into the training objective.


In [32]:
# ==================== MODEL ARCHITECTURES ====================

# ==================== IMPROVED PINN–KAN ARCHITECTURE ====================
class PINN_KAN(nn.Module):
    """
    Research-grade Physics-Informed Kolmogorov–Arnold Network:
        - RBF + linear KAN layers
        - LayerNorm for stable PDE training
        - Tanh activation for smooth second derivatives
        - Skip-connection (KAN-style functional bypass)
        - More expressive RBF dimensions
    """

    def __init__(self, 
                 input_dim: int = 2, 
                 num_rbfs_list: List[int] = [32, 48, 32], 
                 out_dim: int = 1):

        super().__init__()

        layers = []
        in_dim = input_dim

        # Build KAN stacked layers
        for num_rbfs in num_rbfs_list:
            layers.append(KANLayer(
                input_dim=in_dim,
                num_rbfs=num_rbfs,
                output_dim=num_rbfs       # hidden_dim = number of RBFs
            ))
            layers.append(nn.LayerNorm(num_rbfs))  # stabilizes PDE gradients
            layers.append(nn.Tanh())               # smooth activation for u_xx
            in_dim = num_rbfs

        # Output layer
        layers.append(nn.Linear(in_dim, out_dim))

        # Assemble model
        self.model = nn.Sequential(*layers)

        # Skip connection: improves gradient flow & functional smoothness
        self.skip = nn.Linear(input_dim, out_dim, bias=False)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass:
        u = F_KAN(x) + α * skip(x)
        """
        return self.model(x) + 0.1 * self.skip(x)



class VanillaMLP(nn.Module):
    """Pure data-driven MLP (no physics loss)."""
    def __init__(self, input_dim: int = 2, hidden_dims: List[int] = [64, 64, 32], out_dim: int = 1):
        super().__init__()
        layers = []
        
        layers.append(nn.Linear(input_dim, hidden_dims[0]))
        layers.append(nn.Tanh())
        
        for i in range(len(hidden_dims) - 1):
            layers.append(nn.Linear(hidden_dims[i], hidden_dims[i+1]))
            layers.append(nn.Tanh())
        
        layers.append(nn.Linear(hidden_dims[-1], out_dim))
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)


class VanillaPINN(nn.Module):
    """Vanilla PINN (MLP + Physics loss)."""
    def __init__(self, input_dim: int = 2, hidden_dims: List[int] = [64, 64, 32], out_dim: int = 1):
        super().__init__()
        layers = []
        
        layers.append(nn.Linear(input_dim, hidden_dims[0]))
        layers.append(nn.Tanh())
        
        for i in range(len(hidden_dims) - 1):
            layers.append(nn.Linear(hidden_dims[i], hidden_dims[i+1]))
            layers.append(nn.Tanh())
        
        layers.append(nn.Linear(hidden_dims[-1], out_dim))
        
        self.model = nn.Sequential(*layers)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

print("✅ Model architectures loaded")


✅ Model architectures loaded


## Step 3: Physics Residuals

Here we define the Allen–Cahn PDE. In display form:

$$
u_t = \varepsilon^{2}\, u_{xx} - f(u)
$$

Equivalently, the physics residual (which the PINN minimizes) can be written as

$$
\mathcal{R}(x,t) \;=\; u_t(x,t) \;-\; \varepsilon^{2}\,u_{xx}(x,t) \;+\; f\big(u(x,t)\big).
$$

A commonly used choice for the nonlinear reaction term is the double-well potential derivative:

$$
f(u) = u^{3} - u,
$$

so that the PDE becomes

$$
u_t = \varepsilon^{2}\,u_{xx} - (u^{3} - u).
$$

**Purpose:**  
The residual enforces physical consistency — the predicted solution must minimize \(\mathcal{R}(x,t)\) across the spatial–temporal domain.


In [33]:
# ==================== PHYSICS RESIDUALS (IMPROVED) ====================

def allen_cahn_pde_residual(model: nn.Module, x: torch.Tensor, t: torch.Tensor, epsilon: float) -> torch.Tensor:
    """
    Stable Allen–Cahn residual:
        u_t = ε² u_xx - (u³ - u)
    Includes:
        - shape-safe handling for x,t
        - clamped u for stability
        - stabilizer λu term
        - residual clipping
    """

    # Ensure correct shape (N,1)
    if x.dim() == 1:
        x = x.unsqueeze(-1)
    if t.dim() == 1:
        t = t.unsqueeze(-1)

    # Construct input
    X = torch.cat([x, t], dim=1).clone().detach().requires_grad_(True)

    # Forward pass
    u = model(X)
    u = torch.clamp(u, -1.5, 1.5)   # physical stabilization

    # Compute first derivatives
    grads = torch.autograd.grad(
        outputs=u, inputs=X, grad_outputs=torch.ones_like(u),
        create_graph=True, retain_graph=True, only_inputs=True
    )[0]

    u_x = grads[:, 0:1]
    u_t = grads[:, 1:2]

    # Compute second derivative wrt x
    u_xx = torch.autograd.grad(
        outputs=u_x, inputs=X, grad_outputs=torch.ones_like(u_x),
        create_graph=True, retain_graph=True, only_inputs=True
    )[0][:, 0:1]

    # Stabilizer term (crucial for ε = 0.01 stiff PDE)
    stabilizer = 1e-4 * u

    # Allen–Cahn residual
    residual = u_t - (epsilon**2) * u_xx + u**3 - u + stabilizer

    # Clip extreme values for stability
    residual = torch.clamp(residual, -5.0, 5.0)

    return residual

print("✅ Improved Physics residual function loaded")


✅ Improved Physics residual function loaded


## Step 4: Loss Function Definitions
We now define the composite loss that combines:
- **PDE residual loss**
- **Initial condition (IC) loss**
- **Boundary condition (BC) loss**

The total loss guides the model toward both data fidelity and physical correctness.


In [34]:
# ==================== LOSS FUNCTIONS ====================

def initial_condition_loss(model: nn.Module, x_ic: torch.Tensor, u_ic_true: torch.Tensor) -> torch.Tensor:
    """
    MSE loss enforcing u(x,0) = u0(x)
    x_ic: shape (N_ic, 1)
    u_ic_true: shape (N_ic, 1)
    """
    t_ic = torch.zeros_like(x_ic).to(x_ic.device)
    X_ic = torch.cat([x_ic, t_ic], dim=1)
    u_pred = model(X_ic)
    return nn.MSELoss()(u_pred, u_ic_true)


def boundary_condition_loss(model: nn.Module, x_bc: torch.Tensor, t_bc: torch.Tensor, u_bc_true: torch.Tensor) -> torch.Tensor:
    """
    Enforce Dirichlet boundary values: u(x=±L, t) = known (in your data it's 0)
    x_bc: (N_bc,1), t_bc: (N_bc,1), u_bc_true: (N_bc,1)
    """
    X_bc = torch.cat([x_bc, t_bc], dim=1).to(x_bc.device)
    u_pred = model(X_bc)
    return nn.MSELoss()(u_pred, u_bc_true)


def data_loss(model: nn.Module, X: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    """Pure data loss (for Vanilla MLP)."""
    pred = model(X)
    return nn.MSELoss()(pred, y)


# ==================== IMPROVED ADAPTIVE PINN LOSS ====================
def pinn_loss(model: nn.Module, 
              X_data: torch.Tensor, y_data: torch.Tensor,
              X_coll: torch.Tensor, epsilon: float,
              X_ic: Optional[torch.Tensor] = None, 
              y_ic: Optional[torch.Tensor] = None,
              X_bc: Optional[torch.Tensor] = None, 
              y_bc: Optional[torch.Tensor] = None,
              alpha: float = 0.1,          # small data weight
              beta: float = 1.0,           # PDE base weight
              gamma_ic: float = 200.0,     # strong initial condition
              gamma_bc: float = 50.0       # moderate boundary condition
              ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Improved PINN loss:
        - data loss (small)
        - physics residual loss (adaptive growing)
        - IC loss (strong)
        - BC loss (moderate)
        - PDE curriculum (adaptive β)
    """

    # ----------------------------- DATA LOSS -----------------------------
    pred = model(X_data)
    loss_data = nn.MSELoss()(pred, y_data)

    # ----------------------------- PHYSICS LOSS -----------------------------
    x_col = X_coll[:, 0]
    t_col = X_coll[:, 1]
    residual = allen_cahn_pde_residual(model, x_col, t_col, epsilon)
    loss_physics = torch.mean(residual**2)

    # ----------------------------- IC LOSS -----------------------------
    loss_ic = torch.tensor(0.0, device=X_data.device)
    if X_ic is not None and y_ic is not None:
        loss_ic = nn.MSELoss()(model(X_ic), y_ic)

    # ----------------------------- BC LOSS -----------------------------
    loss_bc = torch.tensor(0.0, device=X_data.device)
    if X_bc is not None and y_bc is not None:
        loss_bc = nn.MSELoss()(model(X_bc), y_bc)

    # ----------------------------- ADAPTIVE WEIGHTING -----------------------------
    # Safe global epoch counter (use 0 if not set)
    global epoch_num
    if "epoch_num" not in globals():
        epoch_num = 0

    # Curriculum learning for PDE importance
    pde_weight = beta * (1.0 + 0.001 * epoch_num)

    # ----------------------------- TOTAL LOSS -----------------------------
    total_loss = (
        alpha * loss_data +
        pde_weight * loss_physics +
        gamma_ic * loss_ic +
        gamma_bc * loss_bc
    )

    return total_loss, loss_data, loss_physics, loss_ic, loss_bc



print("✅ Loss functions loaded")


✅ Loss functions loaded


## Step 5: Training and Results Visualization

After defining all components, we train the model using gradient descent.

Below, we visualize:
- The **predicted vs true** Allen–Cahn field,
- The **error map** across the domain,
- And optionally, the **training loss curve** over iterations.

In [35]:
# ==================== TRAINING ====================

def train_vanilla_mlp(model: nn.Module, X_data: torch.Tensor, y_data: torch.Tensor,
                      epochs: int = 2000, lr: float = 1e-3, 
                      print_every: int = 200) -> Dict[str, List[float]]:
    """Train vanilla MLP (data-driven only)."""
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_history = []
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        
        loss = data_loss(model, X_data, y_data)
        loss.backward()
        optimizer.step()
        
        loss_history.append(loss.item())
        
        if (epoch + 1) % print_every == 0 or epoch == 0:
            logger.info(f"Epoch {epoch+1}/{epochs} | Loss: {loss.item():.6e}")
    
    return {'loss': loss_history}


# ==================== IMPROVED TRAINING LOOP (ADAM + LBFGS) ====================
def train_pinn(model: nn.Module, X_data: torch.Tensor, y_data: torch.Tensor,
               X_coll: torch.Tensor, epsilon: float,
               X_ic: Optional[torch.Tensor] = None, y_ic: Optional[torch.Tensor] = None,
               X_bc: Optional[torch.Tensor] = None, y_bc: Optional[torch.Tensor] = None,
               epochs: int = 2000, lr: float = 1e-3,
               alpha: float = 1.0, beta: float = 1.0,
               gamma_ic: float = 10.0, gamma_bc: float = 10.0,
               print_every: int = 200) -> Dict[str, List[float]]:
    """Train PINN with improved Adam + LBFGS optimization."""

    global epoch_num
    epoch_num = 0

    # -------------------- ADAM OPTIMIZATION PHASE --------------------
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    loss_history = []
    data_loss_history = []
    physics_loss_history = []
    ic_loss_history = []
    bc_loss_history = []

    print("\n=== Starting Adam optimizer phase ===")

    for epoch in range(epochs):
        epoch_num = epoch  # used by adaptive PDE weighting in pinn_loss()

        optimizer.zero_grad()

        total, d_loss, p_loss, ic_loss, bc_loss = pinn_loss(
            model,
            X_data, y_data,
            X_coll, epsilon,
            X_ic=X_ic, y_ic=y_ic,
            X_bc=X_bc, y_bc=y_bc,
            alpha=0.1,    # small supervised weight
            beta=1.0,     # PDE weight (adaptive inside pinn_loss)
            gamma_ic=200.0,
            gamma_bc=50.0
        )

        total.backward()

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)

        optimizer.step()

        # store logs
        loss_history.append(total.item())
        data_loss_history.append(d_loss.item())
        physics_loss_history.append(p_loss.item())
        ic_loss_history.append(ic_loss.item() if isinstance(ic_loss, torch.Tensor) else ic_loss)
        bc_loss_history.append(bc_loss.item() if isinstance(bc_loss, torch.Tensor) else bc_loss)

        if (epoch + 1) % print_every == 0 or epoch == 0:
            print(f"[Adam] Epoch {epoch+1}/{epochs} | "
                  f"Total: {total.item():.3e} | "
                  f"Data: {d_loss.item():.2e} | Physics: {p_loss.item():.2e} | "
                  f"IC: {ic_loss.item() if isinstance(ic_loss, torch.Tensor) else ic_loss:.2e} | "
                  f"BC: {bc_loss.item() if isinstance(bc_loss, torch.Tensor) else bc_loss:.2e}")

    # -------------------- LBFGS REFINEMENT PHASE --------------------
    print("\n=== Starting LBFGS refinement ===")

    lbfgs = torch.optim.LBFGS(model.parameters(), 
                              max_iter=500,
                              tolerance_grad=1e-9,
                              tolerance_change=1e-9,
                              history_size=50)

    def closure():
        lbfgs.zero_grad()
        total, *_ = pinn_loss(
            model,
            X_data, y_data,
            X_coll, epsilon,
            X_ic=X_ic, y_ic=y_ic,
            X_bc=X_bc, y_bc=y_bc,
            alpha=0.1,
            beta=1.0,
            gamma_ic=200.0,
            gamma_bc=50.0
        )
        total.backward()
        return total

    lbfgs.step(closure)
    print("LBFGS complete.\n")

    return {
        "loss": loss_history,
        "data_loss": data_loss_history,
        "physics_loss": physics_loss_history,
        "ic_loss": ic_loss_history,
        "bc_loss": bc_loss_history
    }

    
    loss_history = []
    data_loss_history = []
    physics_loss_history = []
    ic_loss_history = []
    bc_loss_history = []
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        
        total_loss, d_loss, p_loss, ic_loss, bc_loss = pinn_loss(
            model, X_data, y_data, X_coll, epsilon,
            X_ic=X_ic, y_ic=y_ic, X_bc=X_bc, y_bc=y_bc,
            alpha=alpha, beta=beta, gamma_ic=gamma_ic, gamma_bc=gamma_bc
        )
        
        total_loss.backward()
        optimizer.step()
        
        loss_history.append(total_loss.item())
        data_loss_history.append(d_loss.item())
        physics_loss_history.append(p_loss.item())
        ic_loss_history.append(ic_loss.item() if isinstance(ic_loss, torch.Tensor) else float(ic_loss))
        bc_loss_history.append(bc_loss.item() if isinstance(bc_loss, torch.Tensor) else float(bc_loss))
        
        if (epoch + 1) % print_every == 0 or epoch == 0:
            # Use a logger to avoid duplicate prints when notebook cells are re-executed
            logger.info(
                f"Epoch {epoch+1}/{epochs} | Total: {total_loss.item():.6e} | "
                f"Data: {d_loss.item():.6e} | Physics: {p_loss.item():.6e} | "
                f"IC: {ic_loss.item() if isinstance(ic_loss, torch.Tensor) else ic_loss:.6e} | "
                f"BC: {bc_loss.item() if isinstance(bc_loss, torch.Tensor) else bc_loss:.6e}"
            )
    
    return {
        'loss': loss_history,
        'data_loss': data_loss_history,
        'physics_loss': physics_loss_history,
        'ic_loss': ic_loss_history,
        'bc_loss': bc_loss_history
    }


print("✅ Training functions loaded")


✅ Training functions loaded


In [36]:
# ==================== EVALUATION ====================

def compute_metrics(model: nn.Module, X: torch.Tensor, y: torch.Tensor,
                   X_coll: torch.Tensor, epsilon: float) -> Dict[str, float]:
    """Compute comprehensive evaluation metrics."""
    model.eval()
    with torch.no_grad():
        u_pred = model(X)
        
        # Prediction metrics
        mse = torch.mean((u_pred - y)**2)
        rmse = torch.sqrt(mse)
        mae = torch.mean(torch.abs(u_pred - y))
        rel_l2_error = torch.norm(u_pred - y) / torch.norm(y)
    
    # Physics residual metrics (requires gradients)
    X_coll_eval = X_coll.clone().detach().requires_grad_(True)
    x_col = X_coll_eval[:, 0]
    t_col = X_coll_eval[:, 1]
    
    residual = allen_cahn_pde_residual(model, x_col, t_col, epsilon)
    
    residual_l2_norm = torch.norm(residual, p=2)
    residual_l2_normalized = residual_l2_norm / np.sqrt(len(residual))
    max_residual = torch.max(torch.abs(residual))
    mean_residual = torch.mean(torch.abs(residual))
    
    metrics = {
        'RMSE': rmse.item(),
        'MAE': mae.item(),
        'Relative_L2_Error': rel_l2_error.item(),
        'Residual_L2_Norm': residual_l2_normalized.item(),
        'Max_Residual': max_residual.item(),
        'Mean_Residual': mean_residual.item()
    }
    
    return metrics

print("✅ Evaluation functions loaded")


✅ Evaluation functions loaded


## Step 7: Visualization of Results

Here, we visualize:
- The predicted field `u_pred(x,t)` over the domain,
- The true/reference field `u_true(x,t)`, and
- The absolute error distribution `|u_pred - u_true|`.

These visualizations help confirm if the learned dynamics replicate the Allen–Cahn diffusion-reaction pattern formation.




In [37]:
# ==================== VISUALIZATION ====================

def plot_prediction_vs_actual(model: nn.Module, X: torch.Tensor, y: torch.Tensor,
                              title: str = "Predictions vs Actual"):
    """Scatter plot of predictions vs actual values."""
    model.eval()
    with torch.no_grad():
        preds = model(X).cpu().numpy()
        actual = y.cpu().numpy()
    
    plt.figure(figsize=(8, 6))
    plt.scatter(actual, preds, alpha=0.5)
    plt.xlabel('Actual u', fontsize=12)
    plt.ylabel('Predicted u', fontsize=12)
    plt.title(title, fontsize=14, fontweight='bold')
    plt.plot([actual.min(), actual.max()], [actual.min(), actual.max()], 'k--')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# ==================== RESIDUAL SURFACE PLOT  ====================
from mpl_toolkits.mplot3d import Axes3D

def plot_residual_surface(model, X_coll, epsilon):

    x = X_coll[:, 0].detach().cpu().numpy()
    t = X_coll[:, 1].detach().cpu().numpy()

    res = allen_cahn_pde_residual(
        model,
        X_coll[:, 0],
        X_coll[:, 1],
        epsilon
    ).detach().cpu().numpy()

    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111, projection='3d')

    ax.scatter(x, t, np.abs(res), s=2, c=np.abs(res), cmap='hot')
    ax.set_xlabel("x")
    ax.set_ylabel("t")
    ax.set_zlabel("|Residual|")
    ax.set_title("PDE Residual Surface")

    plt.show()


def plot_solution_heatmaps(model: nn.Module, X: torch.Tensor, y: torch.Tensor,
                          title_prefix: str = ""):
    """Plot predicted, true, and error heatmaps."""
    model.eval()
    with torch.no_grad():
        u_pred = model(X).cpu().numpy()
    
    # Extract coordinates
    x_vals = X[:, 0].cpu().numpy()
    t_vals = X[:, 1].cpu().numpy()
    x_unique = np.unique(x_vals)
    t_unique = np.unique(t_vals)
    
    # Create grids
    X_grid, T_grid = np.meshgrid(x_unique, t_unique)
    U_pred_grid = u_pred.reshape(len(t_unique), len(x_unique))
    U_true_grid = y.cpu().numpy().reshape(len(t_unique), len(x_unique))
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Predicted solution
    im1 = axes[0].contourf(X_grid, T_grid, U_pred_grid, levels=50, cmap='viridis')
    axes[0].set_title(f'{title_prefix} Predicted Solution u(x,t)', 
                      fontsize=14, fontweight='bold')
    axes[0].set_xlabel('x', fontsize=12)
    axes[0].set_ylabel('t', fontsize=12)
    plt.colorbar(im1, ax=axes[0], label='u')
    
    # True solution
    im2 = axes[1].contourf(X_grid, T_grid, U_true_grid, levels=50, cmap='viridis')
    axes[1].set_title('True Solution u(x,t)', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('x', fontsize=12)
    axes[1].set_ylabel('t', fontsize=12)
    plt.colorbar(im2, ax=axes[1], label='u')
    
    # Absolute error
    error = np.abs(U_pred_grid - U_true_grid)
    im3 = axes[2].contourf(X_grid, T_grid, error, levels=50, cmap='hot')
    axes[2].set_title(f'{title_prefix} Absolute Error', 
                      fontsize=14, fontweight='bold')
    axes[2].set_xlabel('x', fontsize=12)
    axes[2].set_ylabel('t', fontsize=12)
    plt.colorbar(im3, ax=axes[2], label='Error')
    
    plt.tight_layout()
    plt.show()


def plot_residual_heatmap(model: nn.Module, X_coll: torch.Tensor, epsilon: float,
                         title_prefix: str = ""):
    """Plot physics residual heatmap."""
    X_coll_eval = X_coll.clone().detach().requires_grad_(True)
    
    model.eval()
    x_col = X_coll_eval[:, 0]
    t_col = X_coll_eval[:, 1]
    
    residual = allen_cahn_pde_residual(model, x_col, t_col, epsilon)
    
    # Extract coordinates
    x_coll = X_coll[:, 0].cpu().numpy()
    t_coll = X_coll[:, 1].cpu().numpy()
    residual_np = residual.detach().cpu().numpy()
    
    # Interpolate for smooth heatmap
    X_grid, T_grid = np.meshgrid(
        np.linspace(x_coll.min(), x_coll.max(), 100),
        np.linspace(t_coll.min(), t_coll.max(), 100)
    )
    residual_grid = griddata(
        (x_coll, t_coll), 
        residual_np.flatten(), 
        (X_grid, T_grid), 
        method='cubic'
    )
    
    plt.figure(figsize=(10, 6))
    im = plt.contourf(X_grid, T_grid, np.abs(residual_grid), levels=50, cmap='hot')
    plt.colorbar(im, label='|Residual|')
    plt.title(f'{title_prefix} Physics Residual Heatmap', 
              fontsize=14, fontweight='bold')
    plt.xlabel('x', fontsize=12)
    plt.ylabel('t', fontsize=12)
    plt.tight_layout()
    plt.show()
    
    print(f"Max absolute residual: {np.abs(residual_np).max():.6e}")
    print(f"Mean absolute residual: {np.abs(residual_np).mean():.6e}")


def plot_loss_curves(loss_dict: Dict[str, List[float]], title: str = "Training Loss"):
    """Plot training loss curves."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Full training history
    axes[0].plot(loss_dict['loss'], label='Total Loss', linewidth=2)
    if 'data_loss' in loss_dict:
        axes[0].plot(loss_dict['data_loss'], label='Data Loss', linewidth=2, alpha=0.7)
        axes[0].plot(loss_dict['physics_loss'], label='Physics Loss', linewidth=2, alpha=0.7)
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Loss', fontsize=12)
    axes[0].set_title(f'{title} (Log Scale)', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    axes[0].set_yscale('log')
    
    # Last 500 epochs
    n = min(500, len(loss_dict['loss']))
    axes[1].plot(loss_dict['loss'][-n:], label='Total Loss', linewidth=2)
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Loss', fontsize=12)
    axes[1].set_title(f'{title} (Last {n} Epochs)', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


def plot_comparison_bar_chart(results: Dict[str, Dict[str, float]]):
    """Bar chart comparing metrics across models."""
    models = list(results.keys())
    metrics = ['RMSE', 'MAE', 'Relative_L2_Error', 'Max_Residual']
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten()
    
    for idx, metric in enumerate(metrics):
        values = [results[model][metric] for model in models]
        axes[idx].bar(models, values, color=['blue', 'orange', 'green'][:len(models)])
        axes[idx].set_ylabel(metric, fontsize=12)
        axes[idx].set_title(f'{metric} Comparison', fontsize=13, fontweight='bold')
        axes[idx].grid(True, alpha=0.3, axis='y')
        
        for i, v in enumerate(values):
            axes[idx].text(i, v, f'{v:.2e}', ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.show()

print("✅ Visualization functions loaded")


✅ Visualization functions loaded


## Step 8: Main Pipeline Execution

This section orchestrates the entire workflow:
1. Initializes the dataset and domain,
2. Builds the model architecture,
3. Defines loss functions and PDE residuals,
4. Trains the model,
5. Evaluates and visualizes results.

It acts as a single entry point to reproduce all results from scratch, ensuring the process is **modular, repeatable, and scalable**.


In [38]:
# ==================== MAIN PIPELINE ====================

def run_allen_cahn_experiment(
    data_path: str,
    collocation_path: str,
    epsilon: float = 0.01,
    epochs: int = 2000,
    lr: float = 1e-3,
    alpha: float = 1.0,
    beta: float = 1.0,
    save_dir: str = 'allen_cahn_results'
) -> Dict[str, any]:
    """
    Complete experimental pipeline for Allen-Cahn equation.
    
    Returns:
        Dictionary with models, metrics, and loss histories
    """
    
    print("="*70)
    print("ALLEN-CAHN EQUATION PINN-KAN EXPERIMENT")
    print("="*70)
    
    # Load data
    print("\n1. Loading data...")
    collocation_df = pd.read_csv(collocation_path)
    X_collocation = torch.tensor(collocation_df.values, dtype=torch.float32).to(device)
    
    full_df = pd.read_csv(data_path)
    X_full = torch.tensor(full_df[['x', 't']].values, dtype=torch.float32).to(device)
    y_full = torch.tensor(full_df['u'].values, dtype=torch.float32).unsqueeze(-1).to(device)

    # --- Sanity checks on CSV shapes and column order ---

    # Collocation must have at least 2 columns (x,t)
    assert collocation_df.shape[1] >= 2, \
        "collocation CSV must contain at least two columns [x, t]"

    # Optional: check column names
    if list(collocation_df.columns[:2]) != ['x', 't']:
        logger.warning("Collocation CSV columns not named ['x','t']; "
                    "assuming the first two columns are (x,t).")

    # Full dataset must have x,t,u in correct shape
    assert X_full.shape[1] == 2, \
        "X_full must contain exactly two columns: [x, t]"

    assert y_full.ndim == 2 and y_full.shape[1] == 1, \
        "y_full must be a column vector shaped (N,1)"

    
    print(f"   X_full shape: {X_full.shape}")
    print(f"   y_full shape: {y_full.shape}")
    print(f"   X_collocation shape: {X_collocation.shape}")

    # ================== FEATURE SCALING (HIGHLY IMPORTANT) ======================
    # Scale x and t → [-1, 1]

    x_min, x_max = X_full[:, 0].min(), X_full[:, 0].max()
    t_min, t_max = X_full[:, 1].min(), X_full[:, 1].max()



    
    # Create initial condition dataset (x, t=0)
    x_ic_np = np.unique(full_df['x'].values)  # grid of x from data
    x_ic = torch.tensor(x_ic_np.reshape(-1,1), dtype=torch.float32).to(device)
    y_ic_np = u0(x_ic_np).reshape(-1,1).astype(np.float32)  # use same u0 used to generate data
    y_ic = torch.tensor(y_ic_np, dtype=torch.float32).to(device)

    X_ic = torch.cat([x_ic, torch.zeros_like(x_ic)], dim=1)  # x, t=0

    
    # Create boundary condition dataset (x = min and max, over times)
    t_bc_np = np.unique(full_df['t'].values)
    x_left = np.full_like(t_bc_np, fill_value=full_df['x'].min()).reshape(-1,1)
    x_right = np.full_like(t_bc_np, fill_value=full_df['x'].max()).reshape(-1,1)
    t_bc = t_bc_np.reshape(-1,1).astype(np.float32)

    # left boundary (Dirichlet = 0 in the dataset)
    X_bc_left = torch.tensor(np.hstack([x_left, t_bc]), dtype=torch.float32).to(device)
    y_bc_left = torch.zeros((len(t_bc), 1), dtype=torch.float32, device=device)

    # right boundary
    X_bc_right = torch.tensor(np.hstack([x_right, t_bc]), dtype=torch.float32).to(device)
    y_bc_right = torch.zeros((len(t_bc), 1), dtype=torch.float32, device=device)

    # combine left+right BCs (keep everything as torch tensors)
    X_bc = torch.cat([X_bc_left, X_bc_right], dim=0)
    y_bc = torch.cat([y_bc_left, y_bc_right], dim=0)

    # Note: X_ic provided twice (as X_ic and x_ic in the IC helper), but our train_pinn expects X_ic,y_ic in same shape (N,2) and (N,1)
    X_ic = X_ic.to(device)
    y_ic = y_ic.to(device)

    def scale_X(X):
        Xs = X.clone()
        Xs[:, 0] = 2.0 * (X[:, 0] - x_min) / (x_max - x_min) - 1.0
        Xs[:, 1] = 2.0 * (X[:, 1] - t_min) / (t_max - t_min) - 1.0
        return Xs
    # Scale all domain-dependent tensors
    X_full_s = scale_X(X_full)
    X_coll_s = scale_X(X_collocation)
    X_ic_s   = scale_X(X_ic)
    X_bc_s   = scale_X(X_bc)

    # Initialize models
    print("\n2. Initializing models...")
    models = {
        'PINN-KAN': PINN_KAN(input_dim=2, num_rbfs_list=[30, 40, 30], out_dim=1).to(device),
        'Vanilla-MLP': VanillaMLP(input_dim=2, hidden_dims=[64, 64, 32], out_dim=1).to(device),
        'Vanilla-PINN': VanillaPINN(input_dim=2, hidden_dims=[64, 64, 32], out_dim=1).to(device)
    }
    
    for name, model in models.items():
        n_params = sum(p.numel() for p in model.parameters())
        print(f"   {name}: {n_params:,} parameters")
    
    # Train models
    print("\n3. Training models...")
    loss_histories = {}
    
    for name, model in models.items():
        print(f"\n{'='*70}")
        print(f"Training {name}")
        print(f"{'='*70}")
        
        if name == 'Vanilla-MLP':
            loss_dict = train_vanilla_mlp(model, X_full, y_full, epochs=epochs, lr=lr)
        else:
            # For PINN-type models (Vanilla-PINN and PINN-KAN) pass IC/BC
            loss_dict = train_pinn(
                model, X_full_s, y_full, X_collocation, epsilon,
                X_ic=X_ic, y_ic=y_ic, X_bc=X_bc, y_bc=y_bc,
                epochs=epochs, lr=lr, alpha=alpha, beta=beta,
                gamma_ic=200.0,
                gamma_bc=50.0, # start with strong enforcement
                print_every=print_every if 'print_every' in locals() else 200
    )

        
        loss_histories[name] = loss_dict
    
    # Evaluate models
    print("\n4. Evaluating models...")
    all_metrics = {}
    
    for name, model in models.items():
        print(f"\nEvaluating {name}...")
        metrics = compute_metrics(model, X_full, y_full, X_collocation, epsilon)
        all_metrics[name] = metrics
        
        print(f"   RMSE: {metrics['RMSE']:.6e}")
        print(f"   MAE: {metrics['MAE']:.6e}")
        print(f"   Max Residual: {metrics['Max_Residual']:.6e}")
    
    # ======================= RESIDUAL SURFACE PLOTS ==========================
    print("\n=== Plotting Residual Surfaces (For Paper Figures) ===")

    # Only PINN-based models need residual surfaces
    plot_residual_surface(models["PINN-KAN"], X_coll_s, epsilon)
    plot_residual_surface(models["Vanilla-PINN"], X_coll_s, epsilon)

    
    # ======================= DIAGNOSTIC: Gradient Norm ==========================
    def gradient_norm(model):
        total = 0.0
        for p in model.parameters():
            if p.grad is not None:
                total += p.grad.norm().item()
        return total

    print("\n=== Diagnostics: Gradient Norms ===")
    for name, model in models.items():
        print(f"{name:12s}: {gradient_norm(model):.4f}")
    
    # ======================= DIAGNOSTIC: PDE Residual L2 ==========================
    print("\n=== Diagnostics: Physics Residuals ===")
    for name, model in models.items():
        with torch.no_grad():
            x_col = X_coll_s[:, 0]
            t_col = X_coll_s[:, 1]
            res = allen_cahn_pde_residual(model, x_col, t_col, epsilon)
            print(f"{name:12s}: Residual L2 = {torch.norm(res).item():.4e}")


    # Visualizations
    print("\n5. Generating visualizations...")
    
    for name, model in models.items():
        print(f"\nVisualizations for {name}:")
        
        # Prediction vs actual
        plot_prediction_vs_actual(model, X_full, y_full, title=f"{name}: Predictions vs Actual")
        
        # Solution heatmaps
        plot_solution_heatmaps(model, X_full, y_full, title_prefix=name)
        
        # Residual heatmap (only for PINN models)
        if 'PINN' in name or 'KAN' in name:
            plot_residual_heatmap(model, X_collocation, epsilon, title_prefix=name)
        
        # Loss curves
        plot_loss_curves(loss_histories[name], title=f"{name} Training")
    
    # Comparison plot
    print("\nGenerating comparison chart...")
    plot_comparison_bar_chart(all_metrics)
    
    # Save results
    print(f"\n6. Saving results to {save_dir}...")
    os.makedirs(save_dir, exist_ok=True)
    
    # Save models
    for name, model in models.items():
        model_path = os.path.join(save_dir, f"{name.lower().replace('-', '_')}_model.pth")
        torch.save(model.state_dict(), model_path)
    
    # Save metrics
    metrics_df = pd.DataFrame(all_metrics).T
    metrics_df.to_csv(os.path.join(save_dir, 'metrics_comparison.csv'))
    
    with open(os.path.join(save_dir, 'all_results.pkl'), 'wb') as f:
        pickle.dump({
            'metrics': all_metrics,
            'loss_histories': loss_histories
        }, f)
    
    print("\n" + "="*70)
    print("EXPERIMENT COMPLETED!")
    print("="*70)
    
    # Print summary table
    print("\nFINAL METRICS COMPARISON:")
    print(metrics_df.to_string())
    
    return {
        'models': models,
        'metrics': all_metrics,
        'loss_histories': loss_histories
    }

print("✅ Main pipeline loaded")


✅ Main pipeline loaded


In [39]:
def u0(x):
    return x**2 * np.cos(np.pi * x)

## Step 9: Run Experiment

Here, we execute experiments with multiple hyperparameters or configurations (e.g., different network sizes, learning rates, or basis functions).

**Goal:**  
To identify the most accurate and efficient configuration for solving the Allen–Cahn PDE.

Each run logs metrics like:
- Training loss
- Residual loss
- Boundary and initial condition satisfaction
- Total runtime


In [40]:
# ==================== RUN EXPERIMENT ====================

if __name__ == "__main__":
    # Run complete experiment
    results = run_allen_cahn_experiment(
        data_path='../data/allen_cahn_1d.csv',
        collocation_path='../data/allen_cahn_collocation.csv',
        epsilon=0.01,
        epochs=2000,
        lr=1e-3,
        alpha=1.0,
        beta=1.0,
        save_dir='allen_cahn_results'
    )
    
    print("\n✅ All experiments completed successfully!")
    print(f"📊 Results saved in 'allen_cahn_results/' directory")


ALLEN-CAHN EQUATION PINN-KAN EXPERIMENT

1. Loading data...
   X_full shape: torch.Size([51200, 2])
   y_full shape: torch.Size([51200, 1])
   X_collocation shape: torch.Size([20000, 2])

2. Initializing models...
   PINN-KAN: 8,653 parameters
   Vanilla-MLP: 6,465 parameters
   Vanilla-PINN: 6,465 parameters

3. Training models...

Training PINN-KAN

=== Starting Adam optimizer phase ===
[Adam] Epoch 1/2000 | Total: 3.060e+01 | Data: 1.47e-01 | Physics: 8.28e-02 | IC: 1.22e-01 | BC: 1.22e-01


KeyboardInterrupt: 

## Step 10: Diagnostic Analysis and Model Comparison

This diagnostic step compares multiple runs of the PINN–KAN model.

It identifies:
- Best-performing model configurations,
- Error trends,
- Residual convergence behavior.

**Common evaluation metrics:**
- Mean Absolute Error (MAE)
- Root Mean Square Error (RMSE)
- Relative L2 Norm Error
- Physics residual loss


In [ ]:
# Diagnostic comparison cell
# Prints model metrics from `results` produced by the last run and identifies best models
metrics = None
if isinstance(results, dict) and 'metrics' in results:
    metrics = results['metrics']
elif isinstance(results, dict) and all(isinstance(v, dict) for v in results.values()):
    # maybe results already is metrics mapping
    metrics = results
else:
    raise RuntimeError('Could not find metrics in `results` variable')

print('Models found:', list(metrics.keys()))

best_rmse = (None, float('inf'))
best_rel = (None, float('inf'))
best_res = (None, float('inf'))

for name, m in metrics.items():
    print(f"\nModel: {name}")
    for k, v in m.items():
        try:
            print(f"  {k}: {v:.6e}")
        except Exception:
            print(f"  {k}: {v}")
    rmse = m.get('RMSE', float('inf'))
    rel = m.get('Relative_L2_Error', float('inf'))
    res = m.get('Residual_L2_Norm', float('inf'))
    if rmse < best_rmse[1]:
        best_rmse = (name, rmse)
    if rel < best_rel[1]:
        best_rel = (name, rel)
    if res < best_res[1]:
        best_res = (name, res)

print('\nBest by RMSE:', best_rmse)
print('Best by Relative_L2_Error:', best_rel)
print('Best by Residual_L2_Norm:', best_res)


Models found: ['PINN-KAN', 'Vanilla-MLP', 'Vanilla-PINN']

Model: PINN-KAN
  RMSE: 5.732984e-02
  MAE: 3.925442e-02
  Relative_L2_Error: 1.302406e-01
  Residual_L2_Norm: 2.391541e-01
  Max_Residual: 1.021510e+01
  Mean_Residual: 1.023310e-01

Model: Vanilla-MLP
  RMSE: 7.635124e-02
  MAE: 2.489254e-02
  Relative_L2_Error: 1.734527e-01
  Residual_L2_Norm: 8.170205e-02
  Max_Residual: 4.293052e-01
  Mean_Residual: 5.109612e-02

Model: Vanilla-PINN
  RMSE: 1.854281e-01
  MAE: 8.872775e-02
  Relative_L2_Error: 4.212510e-01
  Residual_L2_Norm: 4.359925e-01
  Max_Residual: 9.955010e+00
  Mean_Residual: 1.758494e-01

Best by RMSE: ('PINN-KAN', 0.05732984468340874)
Best by Relative_L2_Error: ('PINN-KAN', 0.1302405595779419)
Best by Residual_L2_Norm: ('Vanilla-MLP', 0.08170205354690552)


# Summary and Conclusion

In this notebook, we implemented and analyzed a **Physics-Informed Neural Network with Kolmogorov–Arnold Network (PINN-KAN)** architecture to solve the **Allen–Cahn equation**.

### Key Takeaways:
- The RBF-based input encoding improves spatial representation.  
- The PINN loss enforces physical consistency via PDE residual minimization.  
- KAN architecture offers interpretability and flexibility for nonlinear PDEs.  
- The model successfully replicates the Allen–Cahn field evolution with low error.

---

### Final Comparative Conclusion

Across all evaluated models, **PINN-KAN** achieved the lowest **RMSE (0.0573)** and **relative L₂ error (0.13)**, confirming that integrating **Kolmogorov–Arnold layers** with **RBF features** enables the model to capture nonlinear dynamics of the Allen–Cahn equation more effectively than both the **Vanilla-MLP** (pure data-driven) and **Vanilla-PINN** (standard physics-informed) baselines.  

Although the Vanilla-MLP exhibited slightly lower residual norms, its higher prediction error indicates that **physics-free models can fit data but fail to generalize** to the full PDE dynamics.  
Hence, **PINN-KAN** provides the best trade-off between **physical consistency** and **predictive accuracy**, demonstrating the benefits of **functional decomposability** and **smooth RBF embeddings** in solving nonlinear PDEs.



---
### Future Work

- Explore adaptive β scheduling for PDE residual weighting.

- Extend to 2D Allen–Cahn or coupled PDEs (e.g., Cahn–Hilliard).

- Compare against Fourier-based PINNs, DeepONets, or Spectral KANs for high-dimensional PDE generalization.

--- 
*This concludes the PINN–KAN Allen–Cahn experiment notebook.*